## Simple Todo App
This tutorial takes you through a step by step look at how to put together a very basic Todo application using [`jupyter-declarativewidgets`](https://github.com/jupyter-incubator/declarativewidgets).

#### Step 1. The Python Code

First, initialize the declarative widgets extension.

In [ ]:
from urth import widgets

widgets.init()

We are going to keep it simple to start with. The Todo `list` will be a Python variable. We will define some simple functions to `add` and `remove` Todo items.

In [ ]:
todo = []

def addTodo(title, date=None):
    global todo
    todo = todo + [(title, date)]

def removeTodo(idx:int):
    global todo
    del todo[idx]

Lets test the code

In [ ]:
addTodo('a')
addTodo('b')
addTodo('c')

In [ ]:
todo

In [ ]:
removeTodo(1)

In [ ]:
todo

#### Step 2. Lets visualize the list
This is where we can use a simple `HTML` template using [Polymer](https://www.polymer-project.org/1.0/) to render the list. The `template` is an `urth-core-bind`, one of the `declarativewidgets` elements, which enhances data binding capabilities in several ways that you will see through the course of this tutorial. In the template we are using the binding variable `{{todo}}` to build the UI and a [dom-repeat](https://www.polymer-project.org/1.0/docs/devguide/templates.html#dom-repeat) `template` to build a simple `HTML` list.

In [ ]:
%%HTML
<template is="urth-core-bind">
    <ul>
        <template is="dom-repeat" items="{{todo}}">
            <li>{{item.0}}</li>
        </template>
    </ul>
</template>

But where are the Todo items?| Well... we need to send the data to the browser so it can be rendered. This is done with the following `Python` code.

In [ ]:
from urth.widgets.widget_channels import channel
channel().set("todo", todo)

Noticed what happened? Now the items of the list are rendered in our HTML template above.

#### Step 3. Lets add to the list
Now we need some UI to add new items to the list. A simple `HTML` form will do, but we need to connect the form submition to a call to the `addTodo` `Python` function. This is done by using the `urth-core-function` element. This element bridges the `HTML` markup and the `Python` code. 

The basics are:
* 'ref' is the name of the function in `Python` (i.e. `addTodo`)
* Parameters for the function are expose in the form `arg-param_name`.
* use `{{}}` template variables to bind values from the form to the function.
* calling the `invoke` method of the `urth-core-function` invokes the `Python` function.

More details on all elements [here](http://jupyter-incubator.github.io/declarativewidgets/docs.html).

In [ ]:
%%HTML
<template is="urth-core-bind">
    <urth-core-function id="addTodo" ref="addTodo" arg-title="{{title}}" arg-date="{{dueDate}}"></urth-core-function>  
    <br/>Add New Item
    <form onSubmit="return false;">
        title: <input id="titleIn" type="text" value="{{title::change}}"></input> 
        date: <input id="dateIn" type="text" value="{{dueDate::change}}"></input> 
        <button onClick="addTodo.invoke(); titleIn.value=''; dateIn.value=''">Add</button>
    </form>
</template>

Try adding something to the Todo list. The cell below show that the items are added.

In [ ]:
todo

Notice that the UI we created above it is not getting updated. The browser has no clue that the `Python` `todo` variable has changes. We need to modify the `addTodo` function to notify that the `todo` list has changed. That is done using `channel().set("todo", todo)`.

In [ ]:
def addTodo(title, date=None):
    global todo
    todo = todo + [(title, date)]
    channel().set("todo", todo)
    
# Only here to refresh the UI    
channel().set("todo", todo)

Now try adding another item and notice that the list ui is not getting updated.

#### Step 3. Lets remove from the list
We will modify our original UI to add a remove button. Now each item will show its content and a button used to remove the item from the Todo list.

In [ ]:
%%HTML
<template is="urth-core-bind">
    <ul>
        <template is="dom-repeat" items="{{todo}}">
            <li>
                <urth-core-function ref="removeTodo" arg-idx="[[index]]" onClick="this.invoke()">
                    <button>x</button> 
                </urth-core-function>  
                {{item.0}}
            </li>
        </template>
    </ul>
</template>

Notice that clicking on the button is not removing the items from the UI. As with `addTod`, we need to modify `removeTodo` to notify that the `todo` list has chanted.

In [ ]:
def removeTodo(idx:int):
    global todo
    del todo[idx]
    channel().set("todo", todo)
    
# Only here to refresh the UI    
channel().set("todo", todo)

Try removing an item now. You should see the list getting updated.

#### Step 4. Lets make it look a bit nicer
With `declarativewidgets` we can import new elements to make our UI look nicer. 

In [ ]:
%%HTML
<link rel='import' href='urth_components/paper-card/paper-card.html' 
        is='urth-core-import' package='PolymerElements/paper-card'>
<link rel='import' href='urth_components/paper-item/paper-item.html' 
        is='urth-core-import' package='PolymerElements/paper-item'>
<link rel='import' href='urth_components/paper-input/paper-input.html' 
        is='urth-core-import' package='PolymerElements/paper-input'>
<link rel='import' href='urth_components/paper-icon-button/paper-icon-button.html' 
        is='urth-core-import' package='PolymerElements/paper-icon-button'>
<link rel='import' href='urth_components/paper-button/paper-button.html' 
        is='urth-core-import' package='PolymerElements/paper-button'>
<link rel='import' href='urth_components/iron-icons/iron-icons.html' 
        is='urth-core-import' package='PolymerElements/iron-icons'>
<link rel='import' href='urth_components/iron-collapse/iron-collapse.html' 
        is='urth-core-import' package='PolymerElements/iron-collapse'>
    
<style is="custom-style">
    paper-card {
        width: 100%;
    }
    
    .list {
        @apply(--layout-vertical);
    }
</style>  

<template is="urth-core-bind">    
    <paper-card heading="Todo List" style="width: 100%">
        <div class="card-content">
            <div class="list" role="listbox">
                <template is="dom-repeat" items="{{todo}}">
                    <paper-item>
                        <urth-core-function ref="removeTodo" arg-idx="[[index]]" onClick="this.invoke()" delay="250">
                            <paper-icon-button icon="clear">X</paper-icon-button>
                        </urth-core-function>  
                        <span>{{item.0}}</span>
                        <template is="dom-if" if="{{item.1}}">
                            <span style="color: gray">&nbsp;({{item.1}})</span>
                        </template>
                    </paper-item>
                </template>
            <div>
        </div>
      <div class="card-actions">
        <paper-button onClick="newItemForm.toggle(); titleIn2.value=''; dueDateIn2.value=''">New Item</paper-button>
        <iron-collapse id="newItemForm">
            <div class="content">
                <paper-input id="titleIn2" label="Title" value="{{title}}"></paper-input>
                <paper-input id="dueDateIn2" label="Due Date" value="{{dueDate}}"></paper-input>
                
                <div style="text-align: right; margin: 10px 20px">
                    <urth-core-function ref="addTodo" arg-title="{{title}}" arg-date="{{dueDate}}" onClick="this.invoke();" delay="250">
                        <paper-button raised onClick="newItemForm.toggle()"><iron-icon icon="add"></iron-icon>Add Item</paper-button>
                    </urth-core-function>
                </div>
            </div>
        </iron-collapse>
      </div>
    </paper-card>    
</template>